In [50]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [76]:
spark = SparkSession.builder \
    .appName('Read Streaming') \
    .master('spark://spark-master:7077') \
    .config("spark.executor.memory", "2g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
    .config('spark.sql.warehouse.dir', f's3a://lakehouse/') \
    .enableHiveSupport() \
    .getOrCreate()

In [52]:
spark

In [27]:
## TODO: create the source for reading stream
lines = spark.readStream.format("socket")\
            .option("host", "160.191.244.13")\
            .option("port", "9999")\
            .load()

In [28]:
## TODO: DataFrane operations
words_df = lines.select(split(col("value"), "\\s").alias("word"))
counts_df = words_df.groupBy("word").count()

In [ ]:
## TODO: create checkpoint folder to save metadata of each batch (stream) -> fault tolerance - consistency (read once)
checkpoint_path = "s3a://lakehouse/streaming/test/checkpoint"

## TODO: create the sink for writing to console
streamingQuery = counts_df.writeStream\
                        .format("console")\
                        .outputMode("complete")\
                        .trigger(processingTime="1 second")\
                        .option("checkpointlocation", checkpoint_path)\
                        .start()

streamingQuery.awaitTermination()

## Write streaming data into Delta

In [ ]:
## TODO: create checkpoint folder to save metadata of each batch (stream) -> fault tolerance - consistency (read once)
checkpoint_path = "s3a://lakehouse/streaming/test/checkpoint"

## TODO: create the sink for writing to console
streamingQuery = counts_df.writeStream\
                        .format("delta")\
                        .outputMode("complete")\
                        .trigger(processingTime="1 second")\
                        .option("path", "s3a://lakehouse/streaming/test/wordcount")
                        .option("checkpointlocation", checkpoint_path)\
                        .start()

streamingQuery.awaitTermination()

- **Load streamed data from delta**

In [77]:
delta_df = spark.read.format("delta").load("s3a://lakehouse/streaming/test/wordcount")
delta_df.show()

+------------+-----+
|        word|count|
+------------+-----+
|[girlfriend]|    1|
| [beautiful]|    1|
|    [friend]|    1|
|      [also]|    1|
|       [are]|    1|
|       [you]|    2|
|        [as]|    1|
|        [my]|    2|
+------------+-----+



In [78]:
spark.stop()